# Black–Litterman para optimización de portafolios

Black–Litterman (BL) es un método para combinar las expectativas del mercado (lo que los precios implican) con **las opiniones (views)** del inversor.

Problema práctico que resuelve:
- Si construyes un portafolio sólo con expectativas históricas, obtendrás estimaciones ruidosas (alta varianza) -> pesos inestables.
- Si sigues ciegamente el mercado (capitalización) también puede no reflejar tus convicciones.

BL mezcla ambos: una *prior* (generalmente la vista del mercado, o "equilibrio") y las *views* del inversor, produciendo rendimientos esperados "posteriores" más robustos y coherentes con tus opiniones.


## Elementos prácticos que necesitas
1. Covarianza de activos (Σ). Puede estimarse desde retornos históricos.
2. Una *prior* de rendimientos: normalmente se usa el rendimiento "implícito" del mercado (equilibrio de mercado) usando el vector de capitalizaciones: el llamado "reverse optimization" donde el rendimiento implícito = δ * Σ * w_market.
   - δ (delta) es el coeficiente de aversión al riesgo (número positivo).

3. Tus *views* (opiniones):
   - Cada view puede ser de forma absoluta (activo A tendrá retorno = 5%) o relativa (A superará a B en 2%).
   - Se codifican en matrices P (matriz de pick) y Q (valores de views).
   
4. Parámetro de incertidumbre de las views (Ω): cuánto confías en cada view.
   Valores pequeños => confianza alta.

## Esquema del procedimiento (práctico)
1. Estimar Σ (covarianza) y escoger δ.
2. Obtener el retorno de equilibrio (π) = δ * Σ * w_market (w_market: pesos de capitalización).
3. Formular views: matrices P y Q.
4. Elegir Ω (matriz diagonal con varianzas asociadas a cada view).
5. Calcular rendimientos posteriori (μ_BL) con la fórmula de Black–Litterman.
6. Usar μ_BL y Σ para optimizar el portafolio (ej. media-varianza simple o Markowitz).

## Fórmulas clave
Prior (equilibrio):
$$
π = δ Σ w_market
$$

Black–Litterman posterior (forma clásica de He and Litterman):
$$
μ_BL = [(τΣ)^{-1} + P^T Ω^{-1} P]^{-1} [(τΣ)^{-1} π + P^T Ω^{-1} Q]
$$

Donde τ es un escalar pequeño que ajusta la incertidumbre en la prior (a menudo 0.025–0.05).
Σ es la matriz de covarianza, P la matriz que selecciona combinaciones de activos,
Q los valores de las views y Ω la incertidumbre de las views.

Luego puedes usar μ_BL en una optimización media-varianza así:
$$
w* = (1/δ) Σ^{-1} μ_BL  (si ignoras restricciones y normalización).
$$

Normalmente escalarás o resolverás una QP para imponer sum(weights)=1, no negativas, etc.

## Consejos prácticos
- τ (tau): no existe consenso; valores típicos 0.02–0.1 o "estimarlo".
- Ω: si no sabes, una regla práctica es usar diag(P τ Σ P^T) * factor (por ejemplo, 0.25).
- Si tus views son muchas y contradictorias, la posterior puede ser inconsistente.
- BL es una forma de *regularizar* tus expectativas: opiniones muy débiles => μ_BL cercano a π.
- Siempre inspecciona la estabilidad de μ_BL ante pequeñas variaciones en Ω y τ.

In [ ]:
# Implementación en Python

import numpy as np
import pandas as pd
from numpy.linalg import inv



# %%
np.random.seed(42)
assets = ["A", "B", "C", "D"]
T = 500  # días
true_means = np.array([0.08, 0.05, 0.12, 0.03]) / 252  # rendimientos diarios verdaderos
true_cov = np.array([[0.10, 0.02, 0.04, 0.01],
                     [0.02, 0.08, 0.01, 0.00],
                     [0.04, 0.01, 0.12, 0.02],
                     [0.01, 0.00, 0.02, 0.06]]) / (252)

# Generar retornos diarios simulados
R = np.random.multivariate_normal(true_means, true_cov, size=T)
returns = pd.DataFrame(R, columns=assets)

# %% [markdown]
# ## 6.2 — Estimar Σ y escoger parámetros

# %%
# Matriz de covarianza histórica
Sigma = returns.cov().values

# Pesos de mercado (ejemplo simple: capitalizaciones hipotéticas)
w_market = np.array([0.4, 0.3, 0.2, 0.1])

# Aversion al riesgo (delta) — regla práctica: 2.5 para horizonte anual; depende de unidad
delta = 2.5

# Tau (incertidumbre de la prior)
tau = 0.05

# Prior de equilibrio (reverse optimization)
pi = delta * Sigma.dot(w_market)

print("Activos:", assets)
print("Sigma estimada:\n", Sigma)
print("Prior (pi) [rendimientos implícitos]:\n", pi)

# %% [markdown]
# ## 6.3 — Definir views (P y Q)
# Ejemplos prácticos:
# 1) View 1 (relativa): A superará a B por 2% anual (≈ 0.02/252 diario)
# 2) View 2 (absoluta): C tendrá un retorno de 10% anual (≈ 0.10/252 diario)

# %%
# Convertir valores anuales a diario (ya trabajamos en diario)
ann_to_daily = 1/252

# P: cada fila es una view; columnas corresponden a activos
P = np.array([[1.0, -1.0, 0.0, 0.0],   # A - B
              [0.0, 0.0, 1.0, 0.0]])  # C

# Q: valores de las views (diarios)
Q = np.array([0.02 * ann_to_daily, 0.10 * ann_to_daily])

# Ω: incertidumbre de las views. Si no sabes, una regla:
# Ω = diag(P (τΣ) P^T) * factor. factor ajusta cuán confiadas son las views.
factor = 0.25
Omega = np.diag(np.diag(P.dot(tau * Sigma).dot(P.T))) * (1.0/factor)

print("P:\n", P)
print("Q:\n", Q)
print("Omega:\n", Omega)

# %% [markdown]
# ## 6.4 — Calcular la posterior BL (μ_BL)

# %%
# Fórmula BL (vectorizado)
tauSigma = tau * Sigma
M_inv = inv(inv(tauSigma) + P.T.dot(inv(Omega)).dot(P))
mu_bl = M_inv.dot(inv(tauSigma).dot(pi) + P.T.dot(inv(Omega)).dot(Q))

print("Rendimientos posterior (mu_bl):\n", mu_bl)

# %% [markdown]
# ## 6.5 — Construir portafolio media-varianza simple
# Sin restricciones (solución analítica): w = (1/delta) Σ^{-1} μ_bl

# %%
w_unconstrained = (1.0/delta) * inv(Sigma).dot(mu_bl)
# Normalizar para que sumen 1 (si se desea)
w_norm = w_unconstrained / np.sum(w_unconstrained)

print("Pesos (no normalizados):\n", w_unconstrained)
print("Pesos normalizados (suman 1):\n", w_norm)
